In [ ]:
%set_env CUDA_VISIBLE_DEVICES=0

# Classification models training

Use this notebook to train classification models (KNN, SVM, etc) on leaf color classification.

## Data gathering

In [ ]:
# Use pre-annotated dataset with each leaf segmentation and class
# From the data.yaml of this dataset, the label number to corresponding class is:
# 0=dark, 1=dead, 2=light, 3=medium

# This creates a list called 'obj_data', which containg every object as a tuple...
# ...containing (obj_classnum, obj_crop)

from typing import List
import scg_detection_tools.utils.image_tools as imtools
import scg_detection_tools.utils.cvt as cvt
from scg_detection_tools.utils.file_handling import get_all_files_from_paths, get_annotation_files
from scg_detection_tools.dataset import read_dataset_annotation
import cv2
import numpy as np
import matplotlib.pyplot as plt

IMG_DIR = "/home/juliocesar/leaf-detection/imgs/light_group/images"
LBL_DIR = "/home/juliocesar/leaf-detection/imgs/light_group/labels"

#IMG_DIR = "/home/juliocesar/leaf-detection/imgs/hemacias/annotated/images"
#LBL_DIR = "/home/juliocesar/leaf-detection/imgs/hemacias/annotated/labels"

imgs = get_all_files_from_paths(IMG_DIR, skip_ext=[".txt", ".json", ".yaml"])

# CHOOSING 32x32 because of calculated average
STANDARD_SIZE = (32, 32)
MAX_MEDIUM_RATIO = 0.30

# !!!!!! taken from data.yaml
class_map = {0: "dark", 1: "dead", 2: "light", 3: "medium"}
#class_map = {0: "purple", 1: "white"}

def extract_objects_from_annotations(imgs: List[str], ann_dir: str, std_size=(32,32), max_cls_ratio: dict[int, float] = None, use_boxes=False):
    img_ann = get_annotation_files(imgs, ann_dir)
    obj_data = []
    sample_count = {cls: 0 for cls in class_map}
    for img in imgs:
        ann_file = img_ann[img]
        annotations = read_dataset_annotation(ann_file, separate_class=False)

        # check if contours are boxes or segments
        orig = cv2.imread(img)
        orig = cv2.cvtColor(orig, cv2.COLOR_BGR2RGB)
        imgsz = orig.shape[:2]

        for ann in annotations:
            nclass = ann[0]
            if nclass in max_cls_ratio:
                if (len(obj_data) >= 1) and ((sample_count[nclass]/len(obj_data)) >= max_cls_ratio[nclass]):
                    continue
            sample_count[nclass] += 1

            contour = ann[1:]
            if use_boxes and len(contour) != 4:
                contour = cvt.segment_to_box(contour, normalized=True, imgsz=imgsz)
            else:
                mask = cvt.contours_to_masks([contour], imgsz=imgsz, normalized=True)[0]
            
            # get only segmented object from image
            if not use_boxes:
                masked = orig.copy()
                masked[mask[:,:] < 255] = 0

                # crop a box around it
                points = np.array(contour).reshape(len(contour) // 2, 2)
                box = cvt.segment_to_box(points, normalized=True, imgsz=imgsz)
                obj_crop = imtools.crop_box_image(masked, box)
            else:
                obj_crop = imtools.crop_box_image(orig, contour)

            # resize to 32x32 and add to our data
            obj_crop = cv2.resize(obj_crop, std_size, cv2.INTER_CUBIC)
            obj_data.append((nclass, obj_crop))
    return obj_data

obj_data = extract_objects_from_annotations(imgs, LBL_DIR, STANDARD_SIZE, max_cls_ratio={3: 0.3}, use_boxes=True)
ncls = [obj[0] for obj in obj_data]
for cls in np.unique(ncls):
    print(f"Samples of type {cls}: {class_map[cls]!r} = {len([c for c in ncls if c == cls])}")


In [29]:
# Split between Train and Test to evaluate model as well
from sklearn.model_selection import train_test_split

X = []
y = []
for nclass, obj_crop in obj_data:
    X.append(obj_crop)
    y.append(nclass)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)
class_labels = [class_map[c] for c in class_map]

In [30]:
# Preprocessing functions (to be able to call clf.predict(imgs) instead of having to extract features first and then calling clf.predict(features))
# -> rn_feature_preprocess: use resnet feature extraction to train classificators
# -> channels_feature_preprocess: extract RGB, HSV and Gray values from a 32x32 image as features
def rn18_feature_preprocess(objX):
    """
    Extract 512-dimensional vector of features from ResNet34
    """
    import numpy as np

    if not isinstance(objX[0], np.ndarray):
        raise TypeError("'objX' passed to preprocess function must be a list of np.ndarray RGB images")

    from object_detection_analysis.classifiers import resnet_extract_features
    processed = []
    for obj in objX:
        processed.append(resnet_extract_features(obj, resnet=18))
    return np.array(processed)

def rn34_feature_preprocess(objX):
    """
    Extract 512-dimensional vector of features from ResNet34
    """
    import numpy as np

    if not isinstance(objX[0], np.ndarray):
        raise TypeError("'objX' passed to preprocess function must be a list of np.ndarray RGB images")

    from object_detection_analysis.classifiers import resnet_extract_features
    processed = []
    for obj in objX:
        processed.append(resnet_extract_features(obj, resnet=34))
    return np.array(processed)

def rn50_feature_preprocess(objX):
    """
    Extract 512-dimensional vector of features from ResNet50
    """
    import numpy as np

    if not isinstance(objX[0], np.ndarray):
        raise TypeError("'objX' passed to preprocess function must be a list of np.ndarray RGB images")

    from object_detection_analysis.classifiers import resnet_extract_features
    processed = []
    for obj in objX:
        processed.append(resnet_extract_features(obj, resnet=50))
    return np.array(processed)

def channels_feature_preprocess(objX):
    """
    Extract RGB, HSV and Gray channels from objects.
    """
    import cv2
    import numpy as np

    if not isinstance(objX[0], np.ndarray):
        raise TypeError("'objX' passed to preprocess function must be a list of np.ndarray RGB images")

    processed = []
    for obj in objX:
        rgb = cv2.resize(obj, (32,32))
        hsv = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)

        attributes = np.concatenate((rgb.flatten(), hsv.flatten(), gray.flatten()))
        processed.append(attributes)

    return np.array(processed)

def norm_channels_feature_preprocess(objX):
    """
    Extract RGB, HSV and Gray channels from objects, normalize each feature vector and then use PCA to reduce dimensionality to 256 features.
    """
    import cv2
    import numpy as np
    from sklearn.preprocessing import StandardScaler
    from sklearn.decomposition import PCA
    from sklearn.pipeline import Pipeline
    
    if not isinstance(objX[0], np.ndarray):
        raise TypeError("'objX' passed to preprocess function must be a list of np.ndarray RGB images")

    processed = []
    for obj in objX:
        rgb = cv2.resize(obj, (32,32))
        hsv = cv2.cvtColor(rgb, cv2.COLOR_RGB2HSV)
        gray = cv2.cvtColor(rgb, cv2.COLOR_RGB2GRAY)

        attributes = np.concatenate((rgb.flatten(), hsv.flatten(), gray.flatten()))
        processed.append(attributes)
    processed = np.array(processed)
    # mean = processed.mean(axis=0, keepdims=True)
    # std = processed.std(axis=0, keepdims=True)
    # norm = (processed - mean) / (std + 1e-7)

    norm_pca_pipe = Pipeline(
        steps = [
            ("scaler", StandardScaler()), 
            #("pca", PCA(n_components=256))
        ]
    )
    norm = norm_pca_pipe.fit_transform(processed)
    return norm.astype(np.float32)


## Parameters test training (e.g. optimal k value for KNN, loss for SGD)

### KNN K

In [ ]:
######################################################
############### K VALUE TEST FOR KNN #################
######################################################

################ CHANNELS FEATURE EXTRACTION
### LAST TESTED OPTIMAL K=2 (no nca)

from object_detection_analysis.classifiers import KNNClassifier

MAX_K = 25
for k in range(1, MAX_K+1):
    knn = KNNClassifier(n_neighbors=k, enable_nca=False, preprocess=channels_feature_preprocess)
    knn.fit(X_train, y_train)
    
    print("_"*82)
    print(f"EVALUATION: K = {k}")
    print("_"*82)
    knn.evaluate(X_test, y_test, disp_labels=class_labels)


In [ ]:
################ RESNET FEATURE EXTRACTION
### BEST FOR RESNET18: K=2
### BEST FOR RESNET34: NONE
### BEST FOR RESNET50: NONE

from object_detection_analysis.classifiers import KNNClassifier

MAX_K = 15
for k in range(1, MAX_K+1):
    knn = KNNClassifier(n_neighbors=k, preprocess=rn34_feature_preprocess, enable_nca=False)
    knn.fit(X_train, y_train)
    
    print("_"*82)
    print(f"EVALUATION: K = {k}")
    print("_"*82)
    knn.evaluate(X_test, y_test, disp_labels=class_labels)

### SGD LOSS

In [ ]:
######################################################
################# LOSS TEST FOR SGD ##################
######################################################

######### CHANNELS FEATURE EXTRACTION
# BEST TEST: LOG_LOSS

from object_detection_analysis.classifiers import SGDBasedClassifier

LOSS_FN = ["hinge", "log_loss", "modified_huber", "squared_hinge", "perceptron", "squared_error", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"]
for loss in LOSS_FN:
    sgd = SGDBasedClassifier(loss=loss, max_iter=1000, preprocess=channels_feature_preprocess)
    sgd.fit(X_train, y_train)

    print("_"*82)
    print(f"EVALUATION: loss = {loss!r}")
    print("_"*82)
    sgd.evaluate(X_test, y_test, disp_labels=class_labels)


In [ ]:
######### RESNET18 FEATURE EXTRACTION
# BEST TEST: HINGE

from object_detection_analysis.classifiers import SGDBasedClassifier

#LOSS_FN = ["hinge", "log_loss", "modified_huber", "squared_hinge", "perceptron", "squared_error", "huber", "epsilon_insensitive", "squared_epsilon_insensitive"]
LOSS_FN = ["hinge"]
for loss in LOSS_FN:
    sgd = SGDBasedClassifier(loss=loss, max_iter=1000, preprocess=rn34_feature_preprocess)
    sgd.fit(X_train, y_train)

    print("_"*82)
    print(f"EVALUATION: loss = {loss!r}")
    print("_"*82)
    sgd.evaluate(X_test, y_test, disp_labels=class_labels)

## Actual training

### KNN

In [32]:
#####################################################
##### TRAIN KNN WITH RESNET FEATURE EXTRACTION #####
#####################################################

from object_detection_analysis.classifiers import KNNClassifier

# LEAF CLASSIFICATION: RESNET18-K=2, RESNET34-K=None, RESNET50-K=None
# BLOOD CELL CLASSIFICATION: K = ?

resnet_knn = KNNClassifier(n_neighbors=2, preprocess=rn18_feature_preprocess)
resnet_knn.fit(X, y)
#resnet_knn.fit(X_train, y_train)
#resnet_knn.evaluate(X_test, y_test, disp_labels=class_labels)

In [33]:
resnet_knn.save_state("knn_rn18_k2_box.skl")

In [34]:
#############################################################
##### TRAIN KNN WITH MANUAL CHANNELS FEATURE EXTRACTION #####
#############################################################

from object_detection_analysis.classifiers import KNNClassifier

# LEAF CLASSIFICATION: K = 2
# BCELL CLASSIFICATION: K = 5

knn = KNNClassifier(n_neighbors=2, preprocess=channels_feature_preprocess)
knn.fit(X, y)
#knn.fit(X_train, y_train)
#knn.evaluate(X_test, y_test, disp_labels=class_labels)

In [35]:
knn.save_state("knn_k2_box.skl")

### SVM

In [36]:
####################################################
##### TRAIN SVM WITH RESNET FEATURE EXTRACTION #####
####################################################

# BEST WITH RESNET18

from object_detection_analysis.classifiers import SVMClassifier

sv = SVMClassifier(preprocess=rn18_feature_preprocess)
sv.fit(X, y)
#sv.fit(X_train, y_train)
#sv.evaluate(X_test, y_test, disp_labels=class_labels)

In [37]:
sv.save_state("svm_rn18_box.skl")

In [38]:
############################################################
##### TRAIN SVM WITH MANUAL CHANNEL FEATURE EXTRACTION #####
############################################################

from object_detection_analysis.classifiers import SVMClassifier

sv = SVMClassifier(preprocess=channels_feature_preprocess)
sv.fit(X, y)
#sv.fit(X_train, y_train)
#sv.evaluate(X_test, y_test, disp_labels=class_labels)

In [39]:
sv.save_state("svm_box.skl")

### SGD

In [19]:
############################################################
##### TRAIN SGD WITH MANUAL CHANNEL FEATURE EXTRACTION #####
############################################################

from object_detection_analysis.classifiers import SGDBasedClassifier

sgd = SGDBasedClassifier(loss="hinge", preprocess=channels_feature_preprocess, max_iter=5000)
sgd.fit(X, y)
#sgd.fit(X_train, y_train)
#sgd.evaluate(X_test, y_test, disp_labels=class_labels)

In [20]:
sgd.save_state("sgd.skl")

In [ ]:
######################################################
##### TRAIN SGD WITH RESNET FEATURE EXTRACTION #####
######################################################

## BEST RESULTS WITH RESNET34

from object_detection_analysis.classifiers import SGDBasedClassifier

sgd = SGDBasedClassifier(loss="hinge", preprocess=rn34_feature_preprocess, max_iter=1000)
sgd.fit(X, y)
#sgd.fit(X_train, y_train)
#sgd.evaluate(X_test, y_test, disp_labels=class_labels)

In [22]:
sgd.save_state("sgd_rn34.skl")

### MLP

In [ ]:
####################################################
##### TRAIN MLP WITH RESNET FEATURE EXTRACTION #####
####################################################

#### BEST RESULTS: RESNET34

from object_detection_analysis.classifiers import MLPClassifier

RN = [rn18_feature_preprocess, rn34_feature_preprocess, rn50_feature_preprocess]
RNID = [18, 34, 50]

for id, func in zip(RNID, RN):
    rn_out_features = 512 if id != 50 else 2048
    mlp = MLPClassifier(n_features=rn_out_features, n_classes=len(class_map), preprocess=func)
    
    mlp.fit(X, y, epochs=50)
    mlp.save_state(f"mlp_rn{id}_box.pt")
    
    # print("_"*82, "\nRESNET:", id, "\n", "_"*82)
    # mlp.fit(X_train, y_train, epochs=50)
    # mlp.evaluate(X_test, y_test, disp_labels=class_labels)
    # print("_"*82)

In [ ]:
#################################################################
##### TRAIN MLP WITH NORMALIZED CHANNELS FEATURE EXTRACTION #####
#################################################################

from object_detection_analysis.classifiers import MLPClassifier

n_features = 32*32*(3 + 3 + 1) # 32x32 leaf RGB, HSV and Gray
mlp = MLPClassifier(n_features=n_features, n_classes=len(class_map), preprocess=norm_channels_feature_preprocess)

mlp.fit(X, y, epochs=50)
#mlp.fit(X_train, y_train, epochs=70)
#mlp.evaluate(X_test, y_test, disp_labels=class_labels)

In [18]:
mlp.save_state("mlp.pt")

## Checking saved states

In [ ]:
###############################################################################################################
## CELLS BELOW ARE FOR CHECKING SAVED MODELS

In [ ]:
from object_detection_analysis.classifiers import KNNClassifier

knn = KNNClassifier.from_state("/home/juliocesar/leaf-detection/checkpoints/classifiers/knn_k4.skl")
knn.evaluate(X_test, y_test, disp_labels=class_labels)

In [ ]:
from object_detection_analysis.classifiers import SVMClassifier

svm = SVMClassifier.from_state("/home/juliocesar/leaf-detection/checkpoints/classifiers/svm.skl")
svm.evaluate(X_test, y_test, disp_labels=class_labels)

In [ ]:
from object_detection_analysis.classifiers import SGDBasedClassifier

sgd = SGDBasedClassifier.from_state("/home/juliocesar/leaf-detection/checkpoints/classifiers/sgd.skl")
sgd.evaluate(X_test, y_test, disp_labels=class_labels)

In [ ]:
from object_detection_analysis.classifiers import MLPClassifier

mlp = MLPClassifier.from_state("/home/juliocesar/leaf-detection/checkpoints/classifiers/mlp.pt")
mlp.evaluate(X_test, y_test, disp_labels=class_labels)

In [ ]:
test_cls, test_obj = obj_data[9]
print(test_cls, test_obj.shape)

mlp.eval()
print(mlp.predict([test_obj]))